Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [X] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [22]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders as ce

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')


In [24]:

def setup_data(target):   #load kaggle data and return 3 way split for target feature
    """take in the target feature, load kaggle and return 3-way split for processing,
    train, val, test,features, target"""

    kaggle_train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                     pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
    test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')

    train, val = train_test_split(kaggle_train, train_size=0.80, test_size=0.20, 
                                  stratify=kaggle_train[target], random_state=8)
    features = train.columns.drop(target) #intial set all features
    return train, val, test, features, target

def wrangle(X): 
    """wrangle single dataframe, return processed copy"""

    X = X.copy()

    #all strings to lower   #needs work
  #  catcols = X.describe(exclude='number').columns
   # for col in catcols:
        #    X[col] = X[col].str.lower()


    X.scheme_management = X.scheme_management.fillna('unknown')
    X.management = X.management.str.lower()
    X.scheme_management = X.scheme_management.str.lower()
    if 'manage' not in X.index:
        X['manage'] = X.management + X.scheme_management
      
       # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    if ('latitude' in X.index):
        X['latitude'] = X['latitude'].replace(-2e-08, 0)
        # When columns have zeros and shouldn't, they are like null values.
        # So we will replace the zeros with nulls, and impute missing values later.
        cols_with_zeros = ['longitude', 'latitude']
        for col in cols_with_zeros:
            X[col] = X[col].replace(0, np.nan)
    if ('permit' in X.index):
        X['permit'] = X['permit'].astype('str')
        X['permit'] = X['permit'].replace({'True': 'yes','False': 'no'})
   
    #X['age'] = pd.DatetimeIndex(X['date_recorded']).year - X.construction_year #not good due to zeros
      # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    
    X['years'] = X['year_recorded'] - X['construction_year']   #age 
    X['years_MISSING'] = X['years'].isnull()     

    dropcols = ['wpt_name',
                'date_recorded',
                'ward','scheme_name',
                'payment_type',     #duplicate of payment
                'quantity_group',   #same as quantity
                'recorded_by',      #1 unique value
                'id']

    for i in dropcols:
        if i in X.columns:
            X.drop(labels=i, axis=1, inplace=True)

    #deal generically with  nans here last resort 
    nancols = X.isnull().sum()[X.isnull().sum() > 0 ].index
    for col in nancols:
        X[col].fillna(value='missing',inplace=True)
    return X

def arrange(df, features):
    """ take in single dataframe, either train or val containing features
    and target, process all columns, and return x and y vectors for features, target lists::
    X_train, y_train = arrange(traindata, featurelist, target) 
    """
    data = wrangle(df) #get new processsed dataframe and column list

    X = data[features]
    y = df[target]
    return X,y

In [31]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV   
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform

train,val,test, allfeatures, target = setup_data('status_group')

catcols = train.select_dtypes(exclude='number')
numcols = train.select_dtypes(include='number').columns.drop('id').tolist()

#features = train.columns.drop([target] + high_cardinality)

card = train.select_dtypes(exclude='number').nunique()   #categoricals
catf = card[card <= 50].index.tolist()

X_train, y_train = arrange(train,features)
X_val, y_val = arrange(val,features)

logreg = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(), 
    StandardScaler(), 
    SelectKBest(), 
    LogisticRegression()
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'selectkbest__k': range(1, len(X_train.columns)+1), 
     'penalty': ['l1', 'l2'],
     'C': uniform(loc=0, scale=4)  #lr hyperparams
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    logreg, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=5, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1

)



NameError: name 'features' is not defined

In [26]:
search.fit(X_train, y_train);

NameError: name 'search' is not defined

In [27]:
# Fit randomized search
best_model = search.fit(X_train, y_train)

NameError: name 'search' is not defined

In [28]:
# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

NameError: name 'best_model' is not defined